# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f47c4ecd100>
├── 'a' --> tensor([[-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477]])
└── 'x' --> <FastTreeValue 0x7f47c4ecdeb0>
    └── 'c' --> tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                        [ 0.2450, -1.0598,  0.5263, -1.1507],
                        [ 0.5405, -0.4624,  1.2087,  0.3226]])

In [4]:
t.a

tensor([[-0.1338, -2.2601,  1.2372],
        [-0.3765,  0.6454,  0.2477]])

In [5]:
%timeit t.a

69.1 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f47c4ecd100>
├── 'a' --> tensor([[ 0.6325,  2.9077,  0.1306],
│                   [-1.6086,  1.3396,  0.7087]])
└── 'x' --> <FastTreeValue 0x7f47c4ecdeb0>
    └── 'c' --> tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                        [ 0.2450, -1.0598,  0.5263, -1.1507],
                        [ 0.5405, -0.4624,  1.2087,  0.3226]])

In [7]:
%timeit t.a = new_value

89.2 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477]]),
    x: Batch(
           c: tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                      [ 0.2450, -1.0598,  0.5263, -1.1507],
                      [ 0.5405, -0.4624,  1.2087,  0.3226]]),
       ),
)

In [10]:
b.a

tensor([[-0.1338, -2.2601,  1.2372],
        [-0.3765,  0.6454,  0.2477]])

In [11]:
%timeit b.a

61.3 ns ± 0.757 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1910, -0.1875,  0.2669],
               [ 0.9606, -1.8527, -0.4199]]),
    x: Batch(
           c: tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                      [ 0.2450, -1.0598,  0.5263, -1.1507],
                      [ 0.5405, -0.4624,  1.2087,  0.3226]]),
       ),
)

In [13]:
%timeit b.a = new_value

578 ns ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

954 ns ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

295 µs ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

266 µs ± 10 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4719b20d30>
├── 'a' --> tensor([[[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]],
│           
│                   [[-0.1338, -2.2601,  1.2372],
│                    [-0.3765,  0.6454,  0.2477]]])
└── 'x' --> <FastTreeValue 0x7f4719b20ac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.2 µs ± 838 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4719b20b80>
├── 'a' --> tensor([[-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477],
│                   [-0.1338, -2.2601,  1.2372],
│                   [-0.3765,  0.6454,  0.2477]])
└── 'x' --> <FastTreeValue 0x7f4720813fd0>
    └── 'c' --> tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                        [ 0.2450, -1.0598,  0.5263, -1.1507],
                 

In [23]:
%timeit t_cat(trees)

39.3 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.3 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]],
       
               [[-0.1338, -2.2601,  1.2372],
                [-0.3765,  0.6454,  0.2477]]]),
    x: Batch(
           c: tensor([[[ 1.2257,  1.3722,  0.4705, -0.4208],
                       [ 0.2450, -1.0598,  0.5263, -1.1507],
                       [ 0.5405, -0.4624,  1.2087,  0.3226]],
         

In [26]:
%timeit Batch.stack(batches)

100 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477],
               [-0.1338, -2.2601,  1.2372],
               [-0.3765,  0.6454,  0.2477]]),
    x: Batch(
           c: tensor([[ 1.2257,  1.3722,  0.4705, -0.4208],
                      [ 0.2450, -1.0598,  0.5263, -1.1507],
                      [ 0.5405, -0.4624,  1.2087,  0.3226],
                      [ 1.2257,  1.3722,  0.4705, -0.4208],
                      [ 0.2450, -

In [28]:
%timeit Batch.cat(batches)

185 µs ± 5.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

587 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
